# Filling positrack gaps with arena_top video

The actuator sometimes comes in the way of the positrack2 cameras during the AutoPI task. To try to eliminate this problem, we can look at the position data extracted from a different video taken from a different camera.

In [1]:
%load_ext autoreload
%autoreload 2
%run setup_project.py
import os.path
from datetime import datetime, timezone
import shutil
import cv2

/home/kilo/repo/analysis_jingjie/jingjie/autopi_mec_jingjie/autopi_mec/mouse_and_objects_position_tracking/setup_project.py:769: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit()
/home/kilo/repo/analysis_jingjie/jingjie/autopi_mec_jingjie/autopi_mec/mouse_and_objects_position_tracking/setup_project.py:797: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit()


In [2]:
projectName, dataPath, dlcModelPath, myProject, sSessions= setup_project_session_lists()

creating myProject, an autopipy.project object
Project name: autopi_mec
dataPath: /adata/projects/autopi_mec
dlcModelPath: /adata/models
Reading /adata/projects/autopi_mec/sessionList_invalid
We have 1 testing sessions in the list
spikeA.Kilosort_session objects are in sSessions


In [3]:
def prepareSessionsForSpatialAnalysisProject(sSessions):
    #print("Creating Animal_pose and Spike_train, sSes.ap and sSes.cg")
    for ses in tqdm(sSessions):
        ses.load_parameters_from_files() 
        ap = Animal_pose(ses)
        ses.ap = ap
        
def load_arena_top_df(ses):
    fn = ses.fileBase+".arena_top_cropped_mouseLEDs_offline_tracking.csv"
    #print("Loading",fn)
    return pd.read_csv(fn)

In [4]:
prepareSessionsForSpatialAnalysisProject(sSessions)

100%|██████████| 1/1 [00:00<00:00, 1069.16it/s]


In [5]:
from scipy.interpolate import interp1d

def mouse_position_fill_gaps(ses):
    """
    Use data from dlc tracking of arena_top.cropped video to get the position of the mouse
    when the mouse was not visible in the positrack2 videos.
    
    only do the x and y at the moment, hd missing
    """
    ses.ap.load_pose_from_file()
    atdf = load_arena_top_df(ses)

    fx = interp1d(atdf.time, atdf.x, bounds_error=False) # x we will start at 0 until the end of the file
    fy = interp1d(atdf.time, atdf.y, bounds_error=False) # y
    
    px = fx(ses.ap.pose[:,-1])
    py = fy(ses.ap.pose[:,-1])
    
    
    
    
    replaceIndicesX = np.logical_and(np.isnan(ses.ap.pose[:,1]), ~np.isnan(px))
    replaceIndicesY = np.logical_and(np.isnan(ses.ap.pose[:,2]), ~np.isnan(py))
    replaceIndices = np.logical_and(replaceIndicesX,replaceIndicesY)
    
    ses.ap.pose[replaceIndices,1] = px[replaceIndices]
    ses.ap.pose[replaceIndices,2] = py[replaceIndices]
    
    # save the new spikeA.Animal_pose.pose array
    
    ses.ap.save_pose_to_file(verbose=False)

In [6]:
ses = sSessions[0]

In [7]:
%%time
mouse_position_fill_gaps(ses)

CPU times: user 208 ms, sys: 92.2 ms, total: 300 ms
Wall time: 1.04 s


In [8]:
for ses in tqdm(sSessions):
    mouse_position_fill_gaps(ses)

100%|██████████| 1/1 [00:01<00:00,  1.04s/it]
